In [1]:
import pandas as pd
from icecream import ic
import spacy
import progressbar
from spacy.matcher import Matcher, DependencyMatcher
# !python -m spacy download en_core_web_sm
nlp=spacy.load("en_core_web_sm")


In [2]:
data=pd.read_csv('dataframe_true_final.csv')
text=data['txt']
data=data.drop(columns=['Unnamed: 0','txt','id'])
text=text.str.replace("'","",regex=False)
text=text.str.replace("]","",regex=False)
text=text.str.replace("[","",regex=False)
text=text.str.replace(",","",regex=False)
text=text.str.replace("A ","a ",regex=False)
text=text.str.replace("And","and",regex=False)

In [92]:
newdata=pd.DataFrame(index=text.index, columns=['PERSON','JOB','NORP','GPE','ORG'])

matcher_nom=DependencyMatcher(nlp.vocab)
pattern_nom=[[{"RIGHT_ID":"nom","RIGHT_ATTRS":{"DEP":"nsubj"}},{"LEFT_ID":"nom","REL_OP":">","RIGHT_ID":"reste","RIGHT_ATTRS":{}}]]
matcher_nom.add("nom",pattern_nom)

matcher_job = Matcher(nlp.vocab)
pattern_job = [[{"DEP": "attr"}]]
matcher_job.add("JOB", pattern_job)


In [93]:
wrong=set()
for i in progressbar.progressbar(range (100)):
  doc=nlp(text[i])
  A=matcher_nom(doc)
  B=matcher_job(doc)
  if A!=[]:   
    newdata.loc[i]['PERSON']=doc[A[0][1][1]:A[0][1][0]+1].text
  else: 
    wrong.add(i)
  if B!=[]:   
    newdata.loc[i]['JOB']=doc[B[0][1]:B[0][2]].text
  else: 
    wrong.add(i)


  for col in ['NORP','GPE','ORG']:
    for ent in doc.ents:
      if ent.label_==col and newdata.iloc[i][col]!=newdata.iloc[i][col]:
        newdata.loc[i][col]=ent.text



for df in [newdata, text]:
  df.drop(wrong).reset_index().drop(columns=['index'])


100% (100 of 100) |######################| Elapsed Time: 0:00:00 Time:  0:00:00


In [7]:
def essai(i): #tester le nouveau dataset
  ic(newdata.iloc[i])
  ic(text[i])
  doc=nlp(text[i])
  for ent in doc.ents:
    ic(ent.text)
    ic(ent.label_)
  for token in doc:
    print(token.text, token.pos_, token.dep_, token.head.text)




In [235]:
essai(470)

ic| newdata.iloc[i]: PERSON    NaN
                     JOB       NaN
                     NORP      NaN
                     GPE       NaN
                     ORG       NaN
                     Name: 470, dtype: object
ic| text[i]: 'Gregor Amann is a German politician and member Of the SPD'
ic| ent.text: 'Gregor Amann'
ic| ent.label_: 'PERSON'
ic| ent.text: 'German'
ic| ent.label_: 'NORP'
ic| ent.text: 'SPD'
ic| ent.label_: 'GPE'
Gregor PROPN compound Amann
Amann PROPN nsubj is
is AUX ROOT is
a DET det politician
German ADJ amod politician
politician NOUN attr is
and CCONJ cc politician
member NOUN conj politician
Of ADP prep politician
the DET det SPD
SPD PROPN pobj Of


In [10]:
# pd.concat([text,newdata],axis=1).to_csv('New_dataset.csv')

In [241]:
# matcher_job=DependencyMatcher(nlp.vocab)
# pattern_job=[[{"RIGHT_ID":"nom","RIGHT_ATTRS":{"DEP":"attr"}},{"LEFT_ID":"nom","REL_OP":">>","RIGHT_ID":"reste","RIGHT_ATTRS":{}}]]
# matcher_job.add("nom",pattern_job)

matcher_job=DependencyMatcher(nlp.vocab)
pattern_job=[[{"RIGHT_ID":"gauche","RIGHT_ATTRS":{}},{"LEFT_ID":"gauche","REL_OP":"<<","RIGHT_ID":"job","RIGHT_ATTRS":{"DEP":"attr"}},{"LEFT_ID":"job","REL_OP":">","RIGHT_ID":"reste","RIGHT_ATTRS":{}}]]
matcher_job.add("nom",pattern_job)



In [245]:
doc=nlp(text[4888])
ic(doc)
A=matcher_job(doc)
long=[]
for i in range(len(A)):
    long.append(max(A[i][1])-min(A[i][1]))
j=long.index(max(long))
ic(doc[min(A[j][1])+1:max(A[j][1])+1])


ic| doc: Denis Oswald is a Swiss rower and sports official
ic| doc[min(A[j][1])+1:max(A[j][1])+1]: Swiss rower and sports


Swiss rower and sports

In [223]:
min(A[7][1]):max(A[7][1])

[3, 7, 11]

In [167]:

Z=doc[i[1][1]:i[1][0]+1]
for j in range(len(Z)):
    ic(j)
    ic(Z[j:j+1])
    if Z[j:j+1].ents!=[]:
        if Z[j:j+1].ents[0].label_=='NORP':
            break

ic(j)

ic| j: 0
ic| Z[j:j+1]: German
ic| j: 0


0